In [2]:
import pandas as pd
import numpy as np 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
#import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import requests

# 2. Data preprocessing

In [3]:
transaction = pd.read_csv('Data\credit_card_transactions-ibm_v2.csv')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Machiavelli127\AppData\Local\Temp\ipykernel_10660\2452642015.py:1: SyntaxWarning: invalid escape sequence '\c'
  transaction = pd.read_csv('Data\credit_card_transactions-ibm_v2.csv')


In [4]:
transaction.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


In [5]:
transaction.tail()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
24386895,1999,1,2020,2,27,22:23,$-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386896,1999,1,2020,2,27,22:24,$54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386897,1999,1,2020,2,28,07:43,$59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No
24386898,1999,1,2020,2,28,20:10,$43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No
24386899,1999,1,2020,2,28,23:10,$45.13,Chip Transaction,4751695835751691036,Merrimack,NH,3054.0,5814,NaN,No


In [6]:
transaction.dtypes.to_frame(name='Type').T.style.set_properties(**{'background-color': 'deepskyblue'})

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
Type,int64,int64,int64,int64,int64,object,object,object,int64,object,object,float64,int64,object,object


In [7]:
transaction['Amount'] = transaction['Amount'].apply(lambda value: value.split("$")[1])
transaction['Hour'] = transaction['Time'].apply(lambda value: value.split(":")[0])
transaction['Minutes'] = transaction['Time'].apply(lambda value: value.split(":")[1])
transaction.drop(['Time'],axis=1,inplace=True)

In [8]:
convert_dict = {'Amount': np.float32,
                'Minutes': np.uint8,
                'Hour': np.uint8,
                'Year': np.int16,
                'Month': np.uint8,
                'Day': np.uint8,
                'Zip': np.float16,
                
                #'MCC':'category',
                #'Zip':'category',
                #'Merchant Name':'category',
                'User':np.uint16,
                'Card':np.uint8
               }

transaction = transaction.astype(convert_dict)

C:\Users\Machiavelli127\AppData\Roaming\Python\Python312\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [9]:
transaction.describe(include='all')

C:\Users\Machiavelli127\AppData\Roaming\Python\Python312\site-packages\pandas\core\nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
C:\Users\Machiavelli127\AppData\Roaming\Python\Python312\site-packages\pandas\core\nanops.py:731: RuntimeWarning: invalid value encountered in scalar divide
  the_mean = the_sum / count if count > 0 else np.nan
C:\Users\Machiavelli127\AppData\Roaming\Python\Python312\site-packages\pandas\core\nanops.py:1013: RuntimeWarning: invalid value encountered in scalar divide
  avg = _ensure_numeric(values.sum(axis=axis, dtype=np.float64)) / count
C:\Users\Machiavelli127\AppData\Roaming\Python\Python312\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Hour,Minutes
count,2.438690e+07,2.438690e+07,2.438690e+07,2.438690e+07,2.438690e+07,2.438690e+07,24386900,2.438690e+07,24386900,21666079,21508765.0,2.438690e+07,388431,24386900,2.438690e+07,2.438690e+07
unique,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,13429,223,NaN,NaN,23,2,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,Swipe Transaction,NaN,ONLINE,CA,NaN,NaN,Insufficient Balance,No,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,15386082,NaN,2720821,2591830,NaN,NaN,242783,24357143,NaN,NaN
mean,1.001019e+03,1.351366e+00,2.011955e+03,6.525064e+00,1.571812e+01,4.363400e+01,NaN,-4.769230e+17,NaN,NaN,NaN,5.561171e+03,NaN,NaN,1.241420e+01,2.958595e+01
std,5.694612e+02,1.407154e+00,5.105921e+00,3.472355e+00,8.794073e+00,8.202238e+01,NaN,4.758940e+18,NaN,NaN,NaN,8.793154e+02,NaN,NaN,5.065882e+00,1.731835e+01
min,0.000000e+00,0.000000e+00,1.991000e+03,1.000000e+00,1.000000e+00,-5.000000e+02,NaN,-9.222899e+18,NaN,NaN,501.0,1.711000e+03,NaN,NaN,0.000000e+00,0.000000e+00
25%,5.100000e+02,0.000000e+00,2.008000e+03,3.000000e+00,8.000000e+00,9.200000e+00,NaN,-4.500543e+18,NaN,NaN,28368.0,5.300000e+03,NaN,NaN,9.000000e+00,1.500000e+01
50%,1.006000e+03,1.000000e+00,2.013000e+03,7.000000e+00,1.600000e+01,3.014000e+01,NaN,-7.946765e+17,NaN,NaN,46752.0,5.499000e+03,NaN,NaN,1.200000e+01,3.000000e+01
75%,1.477000e+03,2.000000e+00,2.016000e+03,1.000000e+01,2.300000e+01,6.506000e+01,NaN,3.189517e+18,NaN,NaN,NaN,5.812000e+03,NaN,NaN,1.600000e+01,4.500000e+01


- Có hơn 24 triệu giao dịch, với 2000 khách hàng và mỗi khách hàng được sở hữu tối đa 9 card.
- Loại giao dịch phổ biến nhất là Swipe và các giao dịch diễn ra phổ biến nhất là ở Olympia, Washington (zip: 98516)
- 

In [10]:
# missing values and proportion
missing_count = transaction.isna().sum()
missing_df = (pd.concat([missing_count.rename('Missing count'), missing_count.div(len(transaction)).rename('Missing ratio')],axis=1).loc[missing_count.ne(0)])
missing_df.style.background_gradient(cmap='Reds')


,Missing count,Missing ratio
Merchant State,2720821,0.111569
Zip,2878135,0.118020
Errors?,23998469,0.984072


In [11]:
#remove transaction fraud
transaction = transaction.loc[transaction['Is Fraud?'] != 'Yes']

In [12]:
transaction.head()

,User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Hour,Minutes
0,0,0,2002,9,1,134.089996,Swipe Transaction,3527213246127876953,La Verne,CA,inf,5300,NaN,No,6,21
1,0,0,2002,9,1,38.480000,Swipe Transaction,-727612092139916043,Monterey Park,CA,inf,5411,NaN,No,6,42
2,0,0,2002,9,2,120.339996,Swipe Transaction,-727612092139916043,Monterey Park,CA,inf,5411,NaN,No,6,22
3,0,0,2002,9,2,128.949997,Swipe Transaction,3414527459579106770,Monterey Park,CA,inf,5651,NaN,No,17,45
4,0,0,2002,9,3,104.709999,Swipe Transaction,5817218446178736267,La Verne,CA,inf,5912,NaN,No,6,23


In [13]:
#drop columns
transaction.drop(['Use Chip','Merchant Name','Merchant City','Zip','MCC','Is Fraud?','Errors?','Merchant State'],axis=1,inplace=True)
transaction.head()

,User,Card,Year,Month,Day,Amount,Hour,Minutes
0,0,0,2002,9,1,134.089996,6,21
1,0,0,2002,9,1,38.480000,6,42
2,0,0,2002,9,2,120.339996,6,22
3,0,0,2002,9,2,128.949997,17,45
4,0,0,2002,9,3,104.709999,6,23


## Users

In [15]:
users_df = pd.read_csv('Data\sd254_users.csv')
users_df.head()

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Machiavelli127\AppData\Local\Temp\ipykernel_10660\729484821.py:1: SyntaxWarning: invalid escape sequence '\s'
  users_df = pd.read_csv('Data\sd254_users.csv')


,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5
1,Sasha Sadr,53,68,1966,12,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,40.76,-73.74,$37891,$77254,$191349,701,5
2,Saanvi Lee,81,67,1938,11,Female,766 Third Drive,NaN,West Covina,CA,91792,34.02,-117.89,$22681,$33483,$196,698,5
3,Everlee Clark,63,63,1957,1,Female,3 Madison Street,NaN,New York,NY,10069,40.71,-73.99,$163145,$249925,$202328,722,4
4,Kyle Peterson,43,70,1976,9,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,37.76,-122.44,$53797,$109687,$183855,675,1


In [16]:
'''users_df.reset_index(inplace=True, drop=False)
users_df.rename(columns={'index':'User'}, inplace=True)
users_df.head()'''

"users_df.reset_index(inplace=True, drop=False)\nusers_df.rename(columns={'index':'User'}, inplace=True)\nusers_df.head()"

In [17]:
#convert data type
users_df['Yearly Income - Person'] = users_df['Yearly Income - Person'].apply(lambda value: value.split("$")[1])
users_df['Total Debt'] = users_df['Total Debt'].apply(lambda value: value.split("$")[1])
convert_dict = {'Yearly Income - Person': np.float32,
                'Total Debt': np.float32,
                'FICO Score': np.int16
               }
users_df = users_df.astype(convert_dict)
users_df.head()

,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,91750,34.15,-117.76,$29278,59696.0,127613.0,787,5
1,Sasha Sadr,53,68,1966,12,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,40.76,-73.74,$37891,77254.0,191349.0,701,5
2,Saanvi Lee,81,67,1938,11,Female,766 Third Drive,NaN,West Covina,CA,91792,34.02,-117.89,$22681,33483.0,196.0,698,5
3,Everlee Clark,63,63,1957,1,Female,3 Madison Street,NaN,New York,NY,10069,40.71,-73.99,$163145,249925.0,202328.0,722,4
4,Kyle Peterson,43,70,1976,9,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,37.76,-122.44,$53797,109687.0,183855.0,675,1
